## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---

## Packages

In [27]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt

## First, I'll compute the camera calibration using chessboard images

In [18]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)

In [21]:
cv2.destroyAllWindows()

### Calculate the disortion coefficients

In [22]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

#### helper to apply the distortion correction to an image

In [24]:
def undistort(img, mtx, dist):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

## Image Pipeline

In [127]:
def add_src_points(img, src):
    """
    print the region of interest on the image - assumes that we're dealing with 4 verticies
    """
    img2 = img
    color = [255, 0, 0]
    thickness = 5
    x0, y0 = src[0]
    x1, y1 = src[1]
    x2, y2 = src[2]
    x3, y3 = src[3]
    cv2.line(img2, (x0, y0), (x1, y1), color, thickness)
    cv2.line(img2, (x1, y1), (x2, y2), color, thickness)
    cv2.line(img2, (x2, y2), (x3, y3), color, thickness)
    cv2.line(img2, (x3, y3), (x0, y0), color, thickness)
    return img2

In [128]:
# Function that runs the pipleline given an image
def binary_pipeline(img, s_thresh=(170, 240), sx_thresh=(15, 90), return_img=""):
    img = np.copy(img)
    
    # Convert to HSV color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))
    
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    
    if return_img == "color":
        return color_binary
    return combined_binary



In [137]:
# Make a list of calibration images
images = glob.glob('../test_images/straight_lines1.jpg')

for fname in images:
    
    img = mpimg.imread(fname)
    img_og = np.copy(img)
    
    undist = undistort(img, mtx, dist)
    
    
    result = binary_pipeline(undist, return_img="color")
    
    imshape = result.shape
    # Points for the persepctive transform
#     src = np.array(
#         [[
#             (275, 675),
#             (600, 450),
#             (680, 450),
#             (1035, 675)
#         ]], dtype=np.int32
#     )
#     dst = np.array(
#         [[
#             (250, 750),
#             (250, 0),
#             (1035, 0),
#             (1035, 750)
#         ]], dtype=np.int32
#     )
    src = np.float32([
        [275, 675],
        [595, 450], 
        [685, 450], 
        [1035, 675]
    ])
    dst = np.float32([
        [250, 750],
        [250, 0], 
        [1035, 0], 
        [1035, 750]
    ])

    src_points_img = add_src_points(img, src)
    
    img_size = (undist.shape[1], undist.shape[0])
    M = cv2.getPerspectiveTransform(src, dst)
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(undist, M, img_size)
    dst_points_warped = add_src_points(warped, dst)

    
### Plot the source points on the original image and the warped image
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(src_points_img)
ax1.set_title('Original Image with Source Points')
ax2.imshow(warped)
ax2.set_title('Warped Perspective')
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    

# image = mpimg.imread(images[0])
# # Plot the result
# f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
# f.tight_layout()

# ax1.imshow(image)
# ax1.set_title('Original Image', fontsize=40)

# ax2.imshow(result)
# ax2.set_title('Pipeline Result', fontsize=40)
# plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)


# print an example plot of a comparison of a distorted and undistorted image
# f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
# f.tight_layout()
# ax1.imshow(img)
# ax1.set_title('Original Image', fontsize=50)
# ax2.imshow(undist)
# ax2.set_title('Undistorted Image', fontsize=50)
# plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [130]:
cv2.destroyAllWindows()

In [87]:
imshape

(720, 1280, 3)

In [124]:
x, y = src[0]

In [125]:
x

275.0

In [126]:
y

675.0